## 初始化

In [20]:
#导入库
import random
import time
import os
import re
import requests
from lxml import etree
import pandas as pd 
import pymysql
import time
import re
from urllib.parse import urlparse,parse_qs,urlencode,urlunparse

MYSQL_HOST = 'localhost'
MYSQL_USER = 'root'
MYSQL_PASSWORD = ''
MYSQL_DATABASE = ''

In [21]:
def get_info(url):

    yushou_cookie = """"""
    headers = {
        "cookie": yushou_cookie,
        "referer": "https://detail.tmall.com/",
        "sec-ch-ua":"""" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90" """,
        "sec-ch-ua-mobile": "?0",
        "sec-fetch-dest": "script",
        "sec-fetch-mode": "no-cors",
        "sec-fetch-site": "cross-site",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36"
    }
    
    r=requests.get(url,headers=headers)
    return r.text

#导出文档
def download_csv(data):
    # 获取采集时间
    open_time = time.localtime(time.time())
    open_time = time.strftime("%Y-%m-%d", open_time) 
    
    #获取文件路径
    curt_path = os.path.abspath("__file__")
    
    #创建文件夹
    folderpath = os.path.dirname(curt_path) + "/" + "预售链接数据源"
    if not os.path.exists(folderpath):
        os.mkdir(folderpath)
        

    #获取文件路径
    #注意是os.path.abspath()!!! os.path.dirname() 返回的是所在文件夹的路径!!
    filname = os.path.abspath(folderpath) + "/" + "预定件数_件单价" + "_" + open_time + ".csv"
    if os.path.exists(filname):
        os.remove(filname)
        
    #导出
    head=['id','时间', '预定件数', '件单价']
    details=pd.DataFrame(columns=head,data=data)
    print(details.info())
    #解决中文乱码
    details.to_csv(filname,encoding = "utf_8_sig")
    print(filname)



In [22]:
class MySQL():
    def __init__(self, host=MYSQL_HOST, username=MYSQL_USER, password=MYSQL_PASSWORD, database=MYSQL_DATABASE):
        """
        MySQL 初始化
        :param host:
        :param username:
        :param password:
        :param port:
        :param database:
        """
        try:
            self.db = pymysql.connect(host=host, user=username, password=password, database=database, charset='utf8')
            self.cursor = self.db.cursor()
        except pymysql.MySQLError as e:
            print(e.args)
            
    def show_tables(self):
        self.cursor.execute('SHOW TABLES')
        tables_list = [i[0] for i in self.cursor.fetchall()]
        return tables_list
    
    def show_rows(self,table):
        sql_rows = "select count(*) from %s;" % (table)
        self.cursor.execute(sql_rows)
        rows = self.cursor.fetchall()
        return rows[0][0]
    
    def delete_id(self,table,data):
        sql_dup = 'SELECT id FROM %s WHERE id=%s' % (table, data)
        res = self.cursor.execute(sql_dup)
        if res>0:  # res为查询到的数据条数如果大于0就代表数据已经存在
            sql_del = 'DELETE FROM %s WHERE id=%s' % (table, data)
            self.cursor.execute(sql_del)
            return 1

    def create_ItemTable(self):
        #建表
        
        #sql_drop = "DROP TABLE IF EXISTS ItemID;"
        #self.cursor.execute(sql_drop)
        
        sql = """
            CREATE TABLE IF NOT EXISTS ItemID(
            id varchar(20) NOT NULL PRIMARY KEY,
            item_link varchar(255) NOT NULL,
            item_name varchar(255) NOT NULL,
            store_name varchar(255) NOT NULL
            ) DEFAULT CHARSET=utf8;
        """
        self.cursor.execute(sql)    
        
        
    #预售链接存入mysql并去重
    def insert_ItemID(self, table, data):
        """
        插入数据
        :param table:
        :param data:
        :return:
        """
        
        head=['id','item_name','item_link','store_name']
        keys = ','.join(head)
        values = ','.join(['%s'] * len(head))
        
        #去重
        sql_dup = 'SELECT id FROM %s WHERE id=%s' % (table, data[0])
        res = self.cursor.execute(sql_dup)
        if res>0:  # res为查询到的数据条数如果大于0就代表数据已经存在
            return 0
        
        #新增链接
        sql_query = 'insert into %s (%s) values(%s)' % (table, keys, values)
        try:
            self.cursor.execute(sql_query, data)
            self.db.commit()
            return 1
        except pymysql.MySQLError as e:
            print(e.args)
            self.db.rollback()
            
    #数据库获取全部预售链接
    def fetch_Item(self):
        sql_items = "SELECT id,item_link FROM ItemID"
        self.cursor.execute(sql_items)
        items = self.cursor.fetchall()
        id_list = []
        item_links = []
        for i in items:
            id_list.append(i[0])
            item_links.append(i[1])
            
        item_dict = dict(zip(id_list,item_links))
        return item_dict
    
    
    #data数据格式(id,item_link,item_name,store_name)
    def extra_ItemID(self, table, data):
        sql_dup = 'SELECT id FROM %s WHERE id=%s' % (table, data[0])
        res = self.cursor.execute(sql_dup)
        if res>0:  # res为查询到的数据条数如果大于0就代表数据已经存在
            return 0
        else:
            #增加预售链接的相关信息
            head=['id','item_link','item_name','store_name']
            keys = ','.join(head)
            values = ','.join(['%s'] * len(head))
            sql_add = 'insert into %s (%s) values(%s)' % (table, keys, values)
            try:
                self.cursor.execute(sql_add, data)
                self.db.commit()
                return 1
            except pymysql.MySQLError as e:
                print(e.args)
                self.db.rollback()        
        

    def create_yushouTable(self,table):
        #创建新表,预售链接的相关信息
        sql = """
            CREATE TABLE IF NOT EXISTS %s(
            id varchar(20) NOT NULL PRIMARY KEY,
            crawl_time DATETIME NOT NULL,
            yushou_order varchar(255) NOT NULL,
            yushou_price varchar(255) NOT NULL
                );
        """ % (table)
        
        self.cursor.execute(sql)
        
    
    def insert_yushou(self,table,data):
        sql_dup = """SELECT id FROM %s WHERE id=%s """ % (table, data[0])
        res = self.cursor.execute(sql_dup)
        #更新
        if res>0:  # res为查询到的数据条数如果大于0就代表数据已经存在
            #sql_update = """UPDATE %s SET yushou_order=%s WHERE id=%s""" % (table,data[2],data[0])
            sql_update = """UPDATE %s 
                SET crawl_time = '%s',
                yushou_order= %s 
                WHERE id=%s""" % (table,data[1],data[2],data[0])
            
            try:
                self.cursor.execute(sql_update)
                self.db.commit()
                return 0
            except pymysql.MySQLError as e:
                print(e.args)
                self.db.rollback()
        else:
            #增加预售链接的相关信息
            head=['id','crawl_time','yushou_order','yushou_price']
            keys = ','.join(head)
            values = ','.join(['%s'] * len(head))
            sql_add = 'insert into %s (%s) values(%s);' % (table, keys, values)
            try:
                self.cursor.execute(sql_add, data)
                self.db.commit()
                return 1
            except pymysql.MySQLError as e:
                print(e.args)
                self.db.rollback()
        
        
    #关闭mysql
    def db_close(self):
        self.cursor.close()
        self.db.close()
            

## 爬取所有预售链接的预定件数

In [23]:
#-------------------执行部分-----------------------
mysql = MySQL()
item_dict = mysql.fetch_Item()
id_list = list(item_dict.keys())
item_links = list(item_dict.values())


print("一共多少个id: ", len(id_list))
download_time = time.localtime(time.time())
download_time = time.strftime("%Y-%m-%d %H:%M:%S", download_time)

start_url = "https://mdskip.taobao.com/core/initItemDetail.htm?isUseInventoryCenter=false&cartEnable=false&service3C=false&isApparel=false&isSecKill=false&tmallBuySupport=true&isAreaSell=true&tryBeforeBuy=false&offlineShop=false&itemId={}&showShopProm=false&isPurchaseMallPage=false&itemGmtModified=1621866591000&isRegionLevel=true&household=false&sellerPreview=false&queryMemberRight=true&addressLevel=3&isForbidBuyItem=false&callback=setMdskip&timestamp={}&isg=eBO5ym9qjT3qEyP_BOfZhurza77TGIRfnuPzaNbMiOCPOY595-bcW6_8U0YpCnhVnsT6R37uITy0BVTskyz67xv9-e6n9MpTld6G.&isg2=BENDsT7GJx7j1uvMADYLahYT0gHtuNf626CebnUg9qIZNGJW_YohS_flr8R6ti_y&cat_id=2&ref=https%3A%2F%2Flist.tmall.com%2F"
urls = [start_url.format(i,str(round(time.time() * 1000))) for i in id_list]


一共多少个id:  1468


In [ ]:
yushou_data = []
error_id = []
count = 0
for url in urls:
    count += 1
    _id = parse_qs(url.split("?")[1])["itemId"][0]
    #随机停顿
    time.sleep(random.uniform(0.2,1))
    try:
        doc = get_info(url)
        yushou_order = int(re.search('(.*)groupUC(.*)maxAmount',doc).group(2).split('"')[2])
        yushou_price = float(re.search('(.*)price(.*)promType',doc).group(2).split('"')[2])
        print(count,_id,item_dict[_id])
        print((_id,download_time,yushou_order,yushou_price))
        yushou_data.append((_id,download_time,yushou_order,yushou_price))
    except Exception as e:
        print("*"*50)
        print("{}: 第{}个预定件数未成功".format(str(_id),str(count)),item_dict[_id])
        print(e)
        error_id.append((count,_id,item_dict[_id]))
        print("*"*50)

In [25]:
len(yushou_data)

660

In [7]:
print("-"*50)
print("开始存入mysql数据库:")
table = "yushou_details_" + time.strftime("%Y%m%d", time.localtime(time.time()))

#先创建新表
tables_list = mysql.show_tables()
if not table in tables_list:
    mysql.create_yushouTable(table)
    print("创建新表:",table)
print("-"*50)

count_num = []
for data in yushou_data:
    print(data,end='\r')
    insert_num = mysql.insert_yushou(table,data)
    count_num.append(insert_num)

insert_num = sum(count_num)
update_num = len(count_num) - sum(count_num)

print('\n')
print("mysql数据表{}:".format(table))
print("合计{}个链接".format(len(count_num)),"插入{}个链接".format(insert_num),"更新{}个链接".format(update_num))
print("-"*50)

--------------------------------------------------
开始存入mysql数据库:
创建新表: yushou_details_20210531
--------------------------------------------------
('7905045388', '2021-05-31 08:53:34', 956, 260.0))0)))

mysql数据表yushou_details_20210531:
合计1455个链接 插入1455个链接 更新0个链接
--------------------------------------------------


In [ ]:
#导出csv
print("导出csv")
download_csv(yushou_data)

In [10]:
#关闭数据库连接
mysql.db_close()

In [30]:
error_id

[(50,
  '610706716420',
  'https://detail.tmall.com/item.htm?id=610706716420&rn=6ce6101c1698e6dcd58aa6f824b62eb3&abbucket=14')]

### 出错id补录入

In [72]:
#使用error_id
error_id

[(154,
  '528897343131',
  'https://detail.tmall.com/item.htm?id=528897343131&rn=d5253fbdb00f889246e24aad81ef90e1&abbucket=14'),
 (417,
  '572644166298',
  'https://detail.tmall.com/item.htm?id=572644166298&rn=eef1bf0f5fb85eb122c1000b565fb35a&abbucket=14'),
 (420,
  '572761535671',
  'https://detail.tmall.com/item.htm?id=572761535671&rn=eef1bf0f5fb85eb122c1000b565fb35a&abbucket=14'),
 (725,
  '613322941433',
  'https://detail.tmall.com/item.htm?id=613322941433&rn=eef1bf0f5fb85eb122c1000b565fb35a&abbucket=14'),
 (1064,
  '633299796393',
  'https://detail.tmall.com/item.htm?id=633299796393&rn=fe81c1ecfa2cae13b7ad079088bef489&abbucket=14')]

In [ ]:
id_list = [i[1] for i in error_id]
item_links = [i[2] for i in error_id]

In [ ]:
item_dict = {}
item_dict = dict(zip(id_list,item_links))

In [73]:
#--------------------------

In [12]:
df = pd.read_excel("工作簿1.xlsx")
df.head()

,店铺,产品,id,是否为彩妆,链接
0,3CE官方旗舰店,3CE丝绒唇釉 哑光雾面口红豆沙南瓜唇彩,587575959944,1,https://detail.tmall.com/item.htm?id=587575959...
1,3CE官方旗舰店,3CE三熹玉柔雾哑光透明口红 雾面持久口红豆沙色,633615409248,1,https://detail.tmall.com/item.htm?id=633615409...
2,3CE官方旗舰店,3CE四色眼影盘 全闪盘金钻杏仁盘彩妆盘珠光便携,628041046276,1,https://detail.tmall.com/item.htm?id=628041046...
3,DIOR迪奥官方旗舰店 香水与美容品,【618预售】全新Dior迪奥烈艳蓝金唇膏口红,643521212193,1,https://detail.tmall.com/item.htm?id=643521212...
4,DIOR迪奥官方旗舰店 香水与美容品,【618预售】全新Dior迪奥烈艳蓝金唇膏传奇丝绒999 720,629792386104,1,https://detail.tmall.com/item.htm?id=629792386...


In [18]:
#-------------------执行部分-----------------------
id_list = df['id'].tolist()
id_list = [str(i) for i in id_list]
item_links = df['链接'].tolist()
item_dict = {}
item_dict = dict(zip(id_list,item_links))

yushou_data = []
error_id = []
count = 0
print("一共多少个id: ", len(id_list))
download_time = time.localtime(time.time())
download_time = time.strftime("%Y-%m-%d", download_time)

start_url = "https://mdskip.taobao.com/core/initItemDetail.htm?isUseInventoryCenter=false&cartEnable=false&service3C=false&isApparel=false&isSecKill=false&tmallBuySupport=true&isAreaSell=true&tryBeforeBuy=false&offlineShop=false&itemId={}&showShopProm=false&isPurchaseMallPage=false&itemGmtModified=1621866591000&isRegionLevel=true&household=false&sellerPreview=false&queryMemberRight=true&addressLevel=3&isForbidBuyItem=false&callback=setMdskip&timestamp={}&isg=eBO5ym9qjT3qEss5BO5Zourza77tNIRb4sPzaNbMiIncC63RT-9tiCxQDCVcnd-RR8XAtDYB4Zk2uL29-etfw_HmndCP97RaxxkDB&isg2=BBISzMflpga2_drDab3qDf8oY970Ixa9h4frJdxrm0Xu77LpxLMzzRlOX0tTn45V&cat_id=2&ref=https%3A%2F%2Flist.tmall.com%2F"
urls = [start_url.format(i,str(round(time.time() * 1000))) for i in id_list]


一共多少个id:  54


In [21]:
for url in urls:
    count += 1
    _id = parse_qs(url.split("?")[1])["itemId"][0]
    #随机停顿
    time.sleep(random.uniform(0.2,1))
    try:
        doc = get_info(url)
        yushou_order = int(re.search('(.*)groupUC(.*)maxAmount',doc).group(2).split('"')[2])
        yushou_price = float(re.search('(.*)price(.*)promType',doc).group(2).split('"')[2])
        print(count,_id,item_dict[_id])
        print((_id,download_time,yushou_order,yushou_price))
        yushou_data.append((_id,download_time,yushou_order,yushou_price))
    except Exception as e:
        print("*"*50)
        print("{}: 第{}个预定件数未成功".format(str(_id),str(count)),e)
        error_id.append((count,_id,item_dict[_id]))
        print("*"*50)

1 587575959944 https://detail.tmall.com/item.htm?id=587575959944&rn=e25b829e14484ad325033c7aab9e7dd3&abbucket=14
('587575959944', '2021-05-30', 47816, 110.0)
2 633615409248 https://detail.tmall.com/item.htm?id=633615409248&rn=e25b829e14484ad325033c7aab9e7dd3&abbucket=14
('633615409248', '2021-05-30', 8204, 125.0)
3 628041046276 https://detail.tmall.com/item.htm?id=628041046276&rn=e25b829e14484ad325033c7aab9e7dd3&abbucket=14
('628041046276', '2021-05-30', 5110, 155.0)
4 643521212193 https://detail.tmall.com/item.htm?id=643521212193&rn=0288e3578d2e5f26ab18c2a2aacaaf96&abbucket=14
('643521212193', '2021-05-30', 6939, 350.0)
5 629792386104 https://detail.tmall.com/item.htm?id=629792386104&rn=0288e3578d2e5f26ab18c2a2aacaaf96&abbucket=14
('629792386104', '2021-05-30', 5638, 350.0)
6 639771136659 https://detail.tmall.com/item.htm?id=639771136659&rn=b643acb66e741e185fdf421319735c55&abbucket=14
('639771136659', '2021-05-30', 203, 530.0)
7 629179077171 https://detail.tmall.com/item.htm?id=629179

53 643892209610 https://detail.tmall.com/item.htm?id=643892209610&rn=ddd0f08f0e5490b91a446fd9ecb36d7f&abbucket=14
('643892209610', '2021-05-30', 53955, 177.0)
54 612648945073 https://detail.tmall.com/item.htm?id=612648945073&rn=ddd0f08f0e5490b91a446fd9ecb36d7f&abbucket=14
('612648945073', '2021-05-30', 69, 139.0)


In [23]:
mysql = MySQL()

In [24]:
print("-"*50)
print("开始存入mysql数据库:")
table = "yushou_details_" + time.strftime("%Y%m%d", time.localtime(time.time()))
count_num = []
#mysql.create_yushouTable(table)
for data in yushou_data:
    print(data,end='\r')
    insert_num = mysql.insert_yushou(table,data)
    count_num.append(insert_num)

insert_num = sum(count_num)
update_num = len(yushou_data) - sum(count_num)

print("mysql数据表{}:".format(table))
print("插入{}个链接".format(insert_num),"更新{}个链接".format(update_num))

--------------------------------------------------
开始存入mysql数据库:
mysql数据表yushou_details_20210530:
插入53个链接 更新0个链接


In [ ]:
#导出csv
print("导出csv")
download_csv(yushou_data)